In [ ]:
import pandas as pd
from openai import OpenAI

In [ ]:
df = pd.read_csv("/content/yelp.csv")[["text", "stars"]].dropna()

test_df = df.sample(n=200, random_state=42)
train_df = df.drop(test_df.index)


In [ ]:
test_df.to_csv("test.csv")

In [ ]:
pip install google-generativeai


In [ ]:
pip install groq

 **PROMPT 1**


In [ ]:
import pandas as pd
import json
import time
from groq import Groq
import re

client = Groq(api_key="api key from gork")
df = pd.read_csv("/content/test (3).csv")

def get_prompt(review_text):
    prompt = f"""Classify this Yelp review into a rating from 1 to 5 stars.

Review: {review_text}

Respond ONLY with valid JSON:
{{"predicted_stars": <number>, "explanation": "<brief reason>"}}"""
    return prompt

# Call API
def classify_review(review_text):
    try:
        prompt = get_prompt(review_text)

        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3
        )

        content = response.choices[0].message.content

        json_match = re.search(r'\{[^{}]*"predicted_stars"[^{}]*\}', content)
        if json_match:
            result = json.loads(json_match.group())
            return result
        else:
            return {"error": "No JSON found", "predicted_stars": None, "explanation": ""}

    except Exception as e:
        return {"error": str(e), "predicted_stars": None, "explanation": ""}

results = []
for idx, row in df.iterrows():
    print(f"Processing {idx+1}/{len(df)}...", end='\r')

    response = classify_review(row['text'])

    results.append({
        "actual_stars": row["stars"],
        "predicted_stars": response.get("predicted_stars"),
        "explanation": response.get("explanation", ""),
        "correct": response.get("predicted_stars") == row["stars"]
    })

    time.sleep(0.1)

results_df = pd.DataFrame(results)

valid_predictions = results_df[results_df['predicted_stars'].notna()]
accuracy = valid_predictions['correct'].sum() / len(valid_predictions) if len(valid_predictions) > 0 else 0
json_validity = len(valid_predictions) / len(results_df)

print(f"\n\nResults:")
print(f"Total Reviews: {len(results_df)}")
print(f"Valid JSON Rate: {json_validity:.2%}")
print(f"Accuracy: {accuracy:.2%}")
print(f"Correct Predictions: {valid_predictions['correct'].sum()}")


results_df.to_csv("classification_results.csv", index=False)
print("\nSaved to: classification_results.csv")



Results:
Total Reviews: 200
Valid JSON Rate: 100.00%
Accuracy: 61.00%
Correct Predictions: 122

Saved to: classification_results.csv


**Prompt 2**

In [ ]:
import pandas as pd
import json
import time
from groq import Groq
import re


client = Groq(api_key="api key from gork")

df = pd.read_csv("/content/test (3).csv")

def get_prompt(review_text):
    prompt = f"""Your task is to analyze restaurant reviews taken from Yelp. Read each review carefully and understand the opinion being expressed. Based on the content, tone, and overall sentiment of the review, predict how many stars the reviewer likely gave. The rating should be a whole number only, from 1 to 5. This is essentially an opinion mining task where you infer the star rating from the written review.
    Use realistic review behavior:
• 5 stars = extremely enthusiastic, glowing praise, strong recommendations, or superlatives
• 4 stars = clearly positive but brief, casual, or focused on a few items without strong emphasis
• 3 stars = mixed or neutral
• 2 stars = mostly negative
• 1 star = very negative or angry"

Review: {review_text}

Respond ONLY with valid JSON:
{{"predicted_stars": <number>, "explanation": "<brief reason>"}}"""
    return prompt


def classify_review(review_text):
    try:
        prompt = get_prompt(review_text)

        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3
        )

        content = response.choices[0].message.content

        json_match = re.search(r'\{[^{}]*"predicted_stars"[^{}]*\}', content)
        if json_match:
            result = json.loads(json_match.group())
            return result
        else:
            return {"error": "No JSON found", "predicted_stars": None, "explanation": ""}

    except Exception as e:
        return {"error": str(e), "predicted_stars": None, "explanation": ""}

results = []
for idx, row in df.iterrows():
    print(f"Processing {idx+1}/{len(df)}...", end='\r')

    response = classify_review(row['text'])

    results.append({
        "actual_stars": row["stars"],
        "review_text":row["text"],
        "predicted_stars": response.get("predicted_stars"),
        "explanation": response.get("explanation", ""),
        "correct": response.get("predicted_stars") == row["stars"]

    })

    time.sleep(0.1)

results_df = pd.DataFrame(results)

valid_predictions = results_df[results_df['predicted_stars'].notna()]
accuracy = valid_predictions['correct'].sum() / len(valid_predictions) if len(valid_predictions) > 0 else 0
json_validity = len(valid_predictions) / len(results_df)

print(f"\n\nResults:")
print(f"Total Reviews: {len(results_df)}")
print(f"Valid JSON Rate: {json_validity:.2%}")
print(f"Accuracy: {accuracy:.2%}")
print(f"Correct Predictions: {valid_predictions['correct'].sum()}")

results_df.to_csv("classification_results.csv", index=False)
print("\nSaved to: classification_results.csv")

Processing 200/200...

Results:
Total Reviews: 200
Valid JSON Rate: 0.50%
Accuracy: 0.00%
Correct Predictions: 0

Saved to: classification_results.csv


**Prompt 3**

In [ ]:
import pandas as pd
import json
import time
from groq import Groq
import re


client = Groq(api_key="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")

df = pd.read_csv("/content/test (3).csv")


def get_prompt(review_text):
    prompt = f"""Your task is to analyze restaurant reviews taken from Yelp. Read each review carefully and understand the opinion being expressed. Based on the content, tone, and overall sentiment of the review, predict how many stars the reviewer likely gave. The rating should be a whole number only, from 1 to 5.

Use realistic review behavior:
• 5 stars = extremely enthusiastic, glowing praise, strong recommendations, or superlatives
• 4 stars = clearly positive but brief, casual, or focused on a few items without strong emphasis
• 3 stars = mixed or neutral
• 2 stars = mostly negative
• 1 star = very negative or angry

Review: {review_text}

You MUST respond with ONLY this exact JSON format (no other text before or after):
{{"predicted_stars": <number>, "explanation": "<brief reason>"}}"""
    return prompt


def classify_review(review_text):
    try:
        prompt = get_prompt(review_text)

        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3,
            max_tokens=150
        )

        content = response.choices[0].message.content.strip()

        print(f"\nRaw response: {content[:200]}")

        try:
            result = json.loads(content)
            if "predicted_stars" in result:
                return result
        except:
            pass

        json_match = re.search(r'```(?:json)?\s*(\{[^`]*\})\s*```', content, re.DOTALL)
        if json_match:
            try:
                result = json.loads(json_match.group(1))
                if "predicted_stars" in result:
                    return result
            except:
                pass


        json_match = re.search(r'\{[^{}]*"predicted_stars"[^{}]*\}', content)
        if json_match:
            try:
                result = json.loads(json_match.group())
                if "predicted_stars" in result:
                    return result
            except:
                pass

        stars_match = re.search(r'"predicted_stars"\s*:\s*(\d)', content)
        if stars_match:
            return {
                "predicted_stars": int(stars_match.group(1)),
                "explanation": "Extracted from non-standard format"
            }

        return {"error": "No valid JSON found", "predicted_stars": None, "explanation": "", "raw": content}

    except Exception as e:
        return {"error": str(e), "predicted_stars": None, "explanation": "", "raw": ""}

print("Testing on first 5 reviews...")
test_results = []
for idx in range(min(5, len(df))):
    row = df.iloc[idx]
    print(f"\n{'='*60}")
    print(f"Review {idx+1}: {row['text'][:100]}...")

    response = classify_review(row['text'])
    print(f"Predicted: {response.get('predicted_stars')}, Actual: {row['stars']}")
    print(f"Explanation: {response.get('explanation', 'N/A')}")
    if 'error' in response:
        print(f"Error: {response['error']}")

    test_results.append(response)

print("\n" + "="*60)
print("If predictions are working, press Enter to continue with full dataset...")
print("If not working, check the raw responses above to debug")
input()

results = []
for idx, row in df.iterrows():
    print(f"Processing {idx+1}/{len(df)}...", end='\r')

    response = classify_review(row['text'])

    results.append({
        "actual_stars": row["stars"],
        "review_text": row["text"],
        "predicted_stars": response.get("predicted_stars"),
        "explanation": response.get("explanation", ""),
        "correct": response.get("predicted_stars") == row["stars"],
        "error": response.get("error", "")
    })

    time.sleep(0.7)

results_df = pd.DataFrame(results)

valid_predictions = results_df[results_df['predicted_stars'].notna()]
accuracy = valid_predictions['correct'].sum() / len(valid_predictions) if len(valid_predictions) > 0 else 0
json_validity = len(valid_predictions) / len(results_df)

print(f"\n\nResults:")
print(f"Total Reviews: {len(results_df)}")
print(f"Valid JSON Rate: {json_validity:.2%}")
print(f"Accuracy: {accuracy:.2%}")
print(f"Correct Predictions: {valid_predictions['correct'].sum()}")

error_count = results_df[results_df['error'] != ''].shape[0]
if error_count > 0:
    print(f"\nErrors encountered: {error_count}")
    print("\nSample errors:")
    print(results_df[results_df['error'] != ''][['error', 'review_text']].head())

results_df.to_csv("classification_results.csv", index=False)
print("\nSaved to: classification_results.csv")

Testing on first 5 reviews...

Review 1: We got here around midnight last Friday... the place was dead. However, they were still serving food...

Raw response: {"predicted_stars": 4, "explanation": "The review is generally positive, mentioning well-made food, friendly service, and a suitable atmosphere, with some neutral comments about the location and late-
Predicted: 4, Actual: 4
Explanation: The review is generally positive, mentioning well-made food, friendly service, and a suitable atmosphere, with some neutral comments about the location and late-night crowd.

Review 2: Brought a friend from Louisiana here.  She says that the crawfish etouffee here is the best she's ha...

Raw response: {"predicted_stars": 5, "explanation": "The reviewer's friend, an out-of-state expert, gave superlative praise to a dish, indicating extremely high satisfaction."}
Predicted: 5, Actual: 5
Explanation: The reviewer's friend, an out-of-state expert, gave superlative praise to a dish, indicating extre

**Prompt 3**

In [ ]:
import pandas as pd
import json
import time
from groq import Groq
import re


client = Groq(api_key="api key from gork")

df = pd.read_csv("/content/test (3).csv")

def get_prompt(review_text):
    prompt = f"""Your task is to Analyze this Yelp review step-by-step. Read each review carefully and understand the opinion being expressed. Based on the content, tone, and overall sentiment of the review, predict how many stars the reviewer likely gave. The rating should be a whole number only, from 1 to 5.

Think through:
1. What positive aspects are mentioned?
2. What negative aspects are mentioned?
3. What is the overall sentiment (very negative, negative, neutral, positive, very positive)?
4. What star rating (1-5) best matches this sentiment?

Review: {review_text}

You MUST respond with ONLY this exact JSON format (no other text before or after):
{{"predicted_stars": <number>, "explanation": "<brief reason>"}}"""
    return prompt
def classify_review(review_text):
    try:
        prompt = get_prompt(review_text)

        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3,
            max_tokens=150
        )

        content = response.choices[0].message.content.strip()


        print(f"\nRaw response: {content[:200]}")

        try:
            result = json.loads(content)
            if "predicted_stars" in result:
                return result
        except:
            pass


        json_match = re.search(r'```(?:json)?\s*(\{[^`]*\})\s*```', content, re.DOTALL)
        if json_match:
            try:
                result = json.loads(json_match.group(1))
                if "predicted_stars" in result:
                    return result
            except:
                pass


        json_match = re.search(r'\{[^{}]*"predicted_stars"[^{}]*\}', content)
        if json_match:
            try:
                result = json.loads(json_match.group())
                if "predicted_stars" in result:
                    return result
            except:
                pass

        stars_match = re.search(r'"predicted_stars"\s*:\s*(\d)', content)
        if stars_match:
            return {
                "predicted_stars": int(stars_match.group(1)),
                "explanation": "Extracted from non-standard format"
            }

        return {"error": "No valid JSON found", "predicted_stars": None, "explanation": "", "raw": content}

    except Exception as e:
        return {"error": str(e), "predicted_stars": None, "explanation": "", "raw": ""}

print("Testing on first 5 reviews...")
test_results = []
for idx in range(min(5, len(df))):
    row = df.iloc[idx]
    print(f"\n{'='*60}")
    print(f"Review {idx+1}: {row['text'][:100]}...")

    response = classify_review(row['text'])
    print(f"Predicted: {response.get('predicted_stars')}, Actual: {row['stars']}")
    print(f"Explanation: {response.get('explanation', 'N/A')}")
    if 'error' in response:
        print(f"Error: {response['error']}")

    test_results.append(response)

print("\n" + "="*60)
print("If predictions are working, press Enter to continue with full dataset...")
print("If not working, check the raw responses above to debug")
input()

results = []
for idx, row in df.iterrows():
    print(f"Processing {idx+1}/{len(df)}...", end='\r')

    response = classify_review(row['text'])

    results.append({
        "actual_stars": row["stars"],
        "review_text": row["text"],
        "predicted_stars": response.get("predicted_stars"),
        "explanation": response.get("explanation", ""),
        "correct": response.get("predicted_stars") == row["stars"],
        "error": response.get("error", "")
    })

    time.sleep(0.7)


results_df = pd.DataFrame(results)


valid_predictions = results_df[results_df['predicted_stars'].notna()]
accuracy = valid_predictions['correct'].sum() / len(valid_predictions) if len(valid_predictions) > 0 else 0
json_validity = len(valid_predictions) / len(results_df)

print(f"\n\nResults:")
print(f"Total Reviews: {len(results_df)}")
print(f"Valid JSON Rate: {json_validity:.2%}")
print(f"Accuracy: {accuracy:.2%}")
print(f"Correct Predictions: {valid_predictions['correct'].sum()}")

error_count = results_df[results_df['error'] != ''].shape[0]
if error_count > 0:
    print(f"\nErrors encountered: {error_count}")
    print("\nSample errors:")
    print(results_df[results_df['error'] != ''][['error', 'review_text']].head())

results_df.to_csv("classification_results.csv", index=False)
print("\nSaved to: classification_results.csv")

Testing on first 5 reviews...

Review 1: We got here around midnight last Friday... the place was dead. However, they were still serving food...

Raw response: {"predicted_stars": 4, "explanation": "The reviewer mentions several positive aspects, including well-made pub grub, friendly service, quality cocktails, and a great atmosphere, with only a minor nega
Predicted: 4, Actual: 4
Explanation: The reviewer mentions several positive aspects, including well-made pub grub, friendly service, quality cocktails, and a great atmosphere, with only a minor negative comment about the place being empty late on a Friday, which is explained by the location.

Review 2: Brought a friend from Louisiana here.  She says that the crawfish etouffee here is the best she's ha...

Raw response: {"predicted_stars": 5, "explanation": "The reviewer's friend, an apparent expert from Louisiana, highly praised the crawfish etouffee, indicating an extremely positive experience."}
Predicted: 5, Actual: 5
Explanatio

In [ ]:
import pandas as pd
import json
import time
from openai import OpenAI
from typing import Dict, List, Tuple
import re
from collections import defaultdict

class YelpReviewClassifier:
    """
    Pipeline for classifying Yelp reviews into 1-5 stars using different prompting approaches.
    """

    def __init__(self, api_key: str):
        """Initialize the classifier with OpenRouter API."""
        self.client = OpenAI(
            base_url="https://openrouter.ai/api/v1",
            api_key=api_key,
        )
        self.model = "tngtech/deepseek-r1t-chimera:free"

    def load_data(self, filepath: str, sample_size: int = 200) -> pd.DataFrame:
        """Load and sample the Yelp dataset."""
        df = pd.read_csv(filepath)
        # Sample stratified by stars to ensure balanced representation
        sampled = df.groupby('stars', group_keys=False).apply(
            lambda x: x.sample(min(len(x), sample_size // 5))
        ).reset_index(drop=True)
        return sampled[:sample_size]

    # ==================== PROMPTING APPROACHES ====================

    def approach_1_direct(self, review_text: str) -> str:
        """
        Approach 1: Direct Classification
        Simple, straightforward instruction with minimal context.
        """
        prompt = f"""Classify this Yelp review into a rating from 1 to 5 stars.

Review: {review_text}

Respond ONLY with valid JSON in this exact format:
{{"predicted_stars": <number>, "explanation": "<brief reason>"}}"""

        return prompt

    def approach_2_detailed_criteria(self, review_text: str) -> str:
        """
        Approach 2: Detailed Criteria-Based
        Provides explicit rating criteria and examples.
        """
        prompt = f"""You are a review analyzer. Classify this Yelp review using these criteria:

Rating Guidelines:
- 5 stars: Exceptional experience, highly enthusiastic, strong recommendation
- 4 stars: Good experience with minor issues, positive overall
- 3 stars: Mixed feelings, acceptable but notable problems
- 2 stars: Poor experience, significant issues, disappointed
- 1 star: Terrible experience, extremely negative, would not return

Review: {review_text}

Analyze the sentiment, specific complaints/praise, and overall tone.
Respond with ONLY valid JSON (no extra text):
{{"predicted_stars": <number>, "explanation": "<brief reason>"}}"""

        return prompt

    def approach_3_chain_of_thought(self, review_text: str) -> str:
        """
        Approach 3: Chain-of-Thought Reasoning
        Encourages step-by-step analysis before classification.
        """
        prompt = f"""Analyze this Yelp review step-by-step and assign a rating from 1-5 stars.

Review: {review_text}

Think through:
1. What positive aspects are mentioned?
2. What negative aspects are mentioned?
3. What is the overall sentiment (very negative, negative, neutral, positive, very positive)?
4. What star rating (1-5) best matches this sentiment?

After your analysis, provide your final answer as valid JSON only:
{{"predicted_stars": <number>, "explanation": "<brief reason>"}}"""

        return prompt

    # ==================== API INTERACTION ====================

    def call_api(self, prompt: str, max_retries: int = 3) -> Dict:
        """Call the OpenRouter API with retry logic."""
        for attempt in range(max_retries):
            try:
                response = self.client.chat.completions.create(
                    model=self.model,
                    messages=[{"role": "user", "content": prompt}],
                    temperature=0.3,  # Lower temperature for more consistent outputs
                )
                content = response.choices[0].message.content
                return self.parse_response(content)

            except Exception as e:
                print(f"API call failed (attempt {attempt + 1}): {str(e)}")
                if attempt < max_retries - 1:
                    time.sleep(2 ** attempt)  # Exponential backoff
                else:
                    return {"error": str(e), "predicted_stars": None, "explanation": "API error"}

        return {"error": "Max retries exceeded", "predicted_stars": None, "explanation": "Failed"}

    def parse_response(self, content: str) -> Dict:
        """Parse API response to extract JSON."""
        try:
            # Try to find JSON in the response
            json_match = re.search(r'\{[^{}]*"predicted_stars"[^{}]*\}', content)
            if json_match:
                result = json.loads(json_match.group())
                # Validate structure
                if "predicted_stars" in result and "explanation" in result:
                    # Ensure predicted_stars is an integer between 1-5
                    stars = int(result["predicted_stars"])
                    if 1 <= stars <= 5:
                        return result

            # If no valid JSON found, return error
            return {
                "error": "Invalid JSON format",
                "predicted_stars": None,
                "explanation": "Could not parse response",
                "raw_response": content
            }

        except (json.JSONDecodeError, ValueError, KeyError) as e:
            return {
                "error": str(e),
                "predicted_stars": None,
                "explanation": "Parse error",
                "raw_response": content
            }

    # ==================== EVALUATION ====================

    def evaluate_approach(self, df: pd.DataFrame, approach_func, approach_name: str) -> Dict:
        """Evaluate a single prompting approach."""
        print(f"\n{'='*60}")
        print(f"Evaluating: {approach_name}")
        print(f"{'='*60}")

        results = []
        valid_json_count = 0
        correct_predictions = 0
        total_processed = 0

        for idx, row in df.iterrows():
            print(f"Processing review {idx + 1}/{len(df)}...", end='\r')

            # Generate prompt and call API
            prompt = approach_func(row['text'])
            response = self.call_api(prompt)

            # Track metrics
            is_valid = "error" not in response
            if is_valid:
                valid_json_count += 1
                predicted = response["predicted_stars"]
                actual = row["stars"]

                if predicted == actual:
                    correct_predictions += 1

                results.append({
                    "review_id": row.get("review_id", idx),
                    "actual_stars": actual,
                    "predicted_stars": predicted,
                    "explanation": response["explanation"],
                    "correct": predicted == actual
                })
            else:
                results.append({
                    "review_id": row.get("review_id", idx),
                    "actual_stars": row["stars"],
                    "predicted_stars": None,
                    "explanation": response.get("explanation", "Error"),
                    "error": response.get("error", "Unknown error"),
                    "correct": False
                })

            total_processed += 1

            # Rate limiting: sleep briefly between requests
            time.sleep(0.5)

        # Calculate metrics
        accuracy = correct_predictions / valid_json_count if valid_json_count > 0 else 0
        json_validity_rate = valid_json_count / total_processed

        metrics = {
            "approach": approach_name,
            "total_reviews": total_processed,
            "valid_json_count": valid_json_count,
            "json_validity_rate": json_validity_rate,
            "correct_predictions": correct_predictions,
            "accuracy": accuracy,
            "results": results
        }

        self.print_metrics(metrics)
        return metrics

    def print_metrics(self, metrics: Dict):
        """Print evaluation metrics."""
        print(f"\n\nResults for {metrics['approach']}:")
        print(f"  Total Reviews: {metrics['total_reviews']}")
        print(f"  Valid JSON Responses: {metrics['valid_json_count']}")
        print(f"  JSON Validity Rate: {metrics['json_validity_rate']:.2%}")
        print(f"  Correct Predictions: {metrics['correct_predictions']}")
        print(f"  Accuracy: {metrics['accuracy']:.2%}")

    def run_full_evaluation(self, df: pd.DataFrame) -> Dict:
        """Run evaluation on all approaches and compare results."""
        approaches = [
            (self.approach_1_direct, "Approach 1: Direct Classification"),
            (self.approach_2_detailed_criteria, "Approach 2: Detailed Criteria"),
            (self.approach_3_chain_of_thought, "Approach 3: Chain-of-Thought")
        ]

        all_metrics = []
        for approach_func, approach_name in approaches:
            metrics = self.evaluate_approach(df, approach_func, approach_name)
            all_metrics.append(metrics)

        # Generate comparison report
        self.generate_comparison_report(all_metrics)

        return {
            "all_metrics": all_metrics,
            "comparison": self.compare_approaches(all_metrics)
        }

    def compare_approaches(self, all_metrics: List[Dict]) -> pd.DataFrame:
        """Create a comparison dataframe of all approaches."""
        comparison_data = []
        for m in all_metrics:
            comparison_data.append({
                "Approach": m["approach"],
                "Accuracy": f"{m['accuracy']:.2%}",
                "JSON Validity": f"{m['json_validity_rate']:.2%}",
                "Correct": m["correct_predictions"],
                "Valid Responses": m["valid_json_count"]
            })

        return pd.DataFrame(comparison_data)

    def generate_comparison_report(self, all_metrics: List[Dict]):
        """Print a comprehensive comparison report."""
        print("\n" + "="*60)
        print("COMPARISON REPORT")
        print("="*60)

        comparison_df = self.compare_approaches(all_metrics)
        print(comparison_df.to_string(index=False))

        # Find best approach
        best_accuracy = max(all_metrics, key=lambda x: x["accuracy"])
        best_validity = max(all_metrics, key=lambda x: x["json_validity_rate"])

        print(f"\nBest Accuracy: {best_accuracy['approach']} ({best_accuracy['accuracy']:.2%})")
        print(f"Best JSON Validity: {best_validity['approach']} ({best_validity['json_validity_rate']:.2%})")

    def save_results(self, results: Dict, output_prefix: str = "yelp_classification"):
        """Save results to CSV files."""
        # Save detailed results for each approach
        for metrics in results["all_metrics"]:
            approach_name = metrics["approach"].replace(" ", "_").replace(":", "")
            results_df = pd.DataFrame(metrics["results"])
            results_df.to_csv(f"{output_prefix}_{approach_name}.csv", index=False)
            print(f"Saved: {output_prefix}_{approach_name}.csv")

        # Save comparison summary
        comparison_df = results["comparison"]
        comparison_df.to_csv(f"{output_prefix}_comparison.csv", index=False)
        print(f"Saved: {output_prefix}_comparison.csv")


# ==================== USAGE EXAMPLE ====================

if __name__ == "__main__":
    # Initialize classifier
    API_KEY = "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"  # Replace with your actual API key
    classifier = YelpReviewClassifier(api_key=API_KEY)

    # Load data
    print("Loading Yelp dataset...")
    df = classifier.load_data("/content/test (3).csv", sample_size=200)
    print(f"Loaded {len(df)} reviews")

    # Run evaluation on all approaches
    print("\nStarting evaluation...")
    results = classifier.run_full_evaluation(df)

    # Save results
    print("\nSaving results...")
    classifier.save_results(results)

    print("\n✓ Evaluation complete!")

Loading Yelp dataset...
Loaded 148 reviews

Starting evaluation...

Evaluating: Approach 1: Direct Classification


/tmp/ipython-input-2619042312.py:26: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled = df.groupby('stars', group_keys=False).apply(


API call failed (attempt 1): Error code: 429 - {'error': {'message': 'Rate limit exceeded: free-models-per-day. Add 10 credits to unlock 1000 free model requests per day', 'code': 429, 'metadata': {'headers': {'X-RateLimit-Limit': '50', 'X-RateLimit-Remaining': '0', 'X-RateLimit-Reset': '1765843200000'}, 'provider_name': None}}, 'user_id': 'user_36qyQBeKAmaQHCcFTtShbvi2xu6'}
API call failed (attempt 2): Error code: 429 - {'error': {'message': 'Rate limit exceeded: free-models-per-day. Add 10 credits to unlock 1000 free model requests per day', 'code': 429, 'metadata': {'headers': {'X-RateLimit-Limit': '50', 'X-RateLimit-Remaining': '0', 'X-RateLimit-Reset': '1765843200000'}, 'provider_name': None}}, 'user_id': 'user_36qyQBeKAmaQHCcFTtShbvi2xu6'}
API call failed (attempt 3): Error code: 429 - {'error': {'message': 'Rate limit exceeded: free-models-per-day. Add 10 credits to unlock 1000 free model requests per day', 'code': 429, 'metadata': {'headers': {'X-RateLimit-Limit': '50', 'X-Rate

KeyboardInterrupt: 